# Standard Model Lagrangian

We will attempt to build all of the Lorentz and gauge invariant terms in the Standard Model Lagrangian. This is done by considering all of the building blocks

$h, \tilde{h}, D_\mu$ have mass dimension 1

$q_L^i, u_R^i, d_R^i, l_L^i, e_R^i$ have mass dimension $\frac{3}{2}$

$G_{\mu \nu}, W_{\mu \nu}, B_{\mu \nu}$ have mass dimension 2

# Building the fields
Would like to add the functionality which allows us to build all the fundamental fields from the groups and representations which define the theory. For now we specify the fundamental fields and how they tranform. 

In [1]:
# Building Blocks
# Each block will have an list of all the meaningful parameters associated with the block
# index
# 0 -> mass dimension
# 1 -> transformation under U(1)
# 2 -> transformation under SU(2)
# 3 -> transformation under SU(3)
# 4 -> spin              
# 5 -> handedness        [m]  U(1)xSU(2)xSU(3) spin hand
blocks2 = {'D':         [1  ,   0 ,  0  ,  0  , 0  ,  0 ], # Covariant derivative should not change how an object transforms.
         'h':           [1  , 1/2 ,  2  ,  1  , 0  ,  0 ],
#        '\\tilde{h}':  [1  ,-1/2 ,  2  ,  1  , 0  ,  0 ],
         'q_L':         [3/2, 1/6 ,  2  ,  3  , 1/2, -1 ], # [u_L^i,d_L^i]
         'u_R':         [3/2, 2/3 ,  1  ,  3  , 1/2,  1 ],
         'd_R':         [3/2,-1/3 ,  1  ,  3  , 1/2,  1 ],
         'l_L':         [3/2,-1/2 ,  2  ,  1  , 1/2, -1 ], # [e_L^i,v_L^i]
         'e_R':         [3/2,  -1 ,  1  ,  1  , 1/2,  1 ],
#        'v_R':         [3/2,   0 ,  1  ,  1  , 1/2,  1 ], # Right handed neutrino might not exist. Or might be dark matter.
         'G_{\mu \\nu}':[2  ,   0 ,  1  ,  8  , 1  ,  0 ], # SU(3) gauge boson will transform trivially under other gauge transformations
         'W_{\mu \\nu}':[2  ,   0 ,  3  ,  1  , 1  ,  0 ], # SU(2) gauge boson will transform trivially under other gauge transformations
         'B_{\mu \\nu}':[2  ,   0 ,  1  ,  1  , 1  ,  0 ]} # U(1) gauge boson will transform trivially under all gauge transformations

In [2]:
#                        [m]  U(1)xSU(2)xSU(3) spin hand
normal = {'D':          [2  ,   0 ,  0  ,  0  , 0  ,  0 ], # Covariant derivative should not change how an object transforms.
         'h':           [2  ,   3 ,  2  ,  1  , 0  ,  0 ],
#        '\\tilde{h}':  [2  ,  -3 ,  2  ,  1  , 0  ,  0 ],
         'q_L^i':       [3  ,   1 ,  2  ,  3  , 1/2, -1 ],# [u_L^i,d_L^i]
         'u_R^i':       [3  ,   4 ,  1  ,  3  , 1/2,  1 ],
         'd_R^i':       [3  ,  -2 ,  1  ,  3  , 1/2,  1 ],
         'l_L^i':       [3  ,  -3 ,  2  ,  1  , 1/2, -1 ],# [e_L^i,v_L^i]
         'e_R^i':       [3  ,  -6 ,  1  ,  1  , 1/2,  1 ],
#        'v_R^i':       [3  ,   0 ,  1  ,  1  , 1/2,  1 ], # Right handed neutrino might not exist. Or might be dark matter.
         'G_{\mu \\nu}':[4  ,   0 ,  1  ,  8  , 1  ,  0 ], # SU(3) gauge boson will transform trivially under other gauge transformations
         'W_{\mu \\nu}':[4  ,   0 ,  3  ,  1  , 1  ,  0 ], # SU(2) gauge boson will transform trivially under other gauge transformations
         'B_{\mu \\nu}':[4  ,   0 ,  1  ,  1  , 1  ,  0 ]} # U(1) gauge boson will transform trivially under all gauge transformations

# Conditions for valid terms

- Need mass dimension less than 4
- Need to be Lorentz invariant (even number of Fermions?)
- Needs to be gauge invariant 
    - Even number of SU(3) indices (if transforms non trivially, has indices)
    - Even number of SU(2) indices (if transforms non trivially, has indices)
    - U(1) charge sums to integer
- Each covariant derivative needs to contract with a (two?) Fermion(s) or another covariant derivative 

# How to find all combinations
- Need a way of handling daggars - conjugation
- Need a way of handling indices

could do a terrible brute force and say that since the most fields a term could have would be 4 since every field has mass dimension at leat 1 and we constrict our terms to be less than or equal to 4

In [3]:
# first check is to see if it has mass dimension less than 4
def validMass(curr,blocks,maxDim):
    massDim = 0
    for t in curr: massDim += blocks[t[0]][0]
    return massDim <= maxDim and massDim%1 == 0

#loop through and add all Lorentz indices, -1 for lower +1 for higher. Needs to equal 0 to be fully contracted
def LorentzIndex(curr):
    lorentzIndex = 0
    for t in curr: lorentzIndex += t[4] 
    return lorentzIndex

#U(1)
def hyperCharge(curr, blocks):
    hyperCharge = 0
    for t in curr: hyperCharge += t[1] * blocks[t[0]][1]
    return hyperCharge

#SU(2)
# loop through and sum all spin indices, -1 for lower +1 for higher. Needs to equal 0 
def weakCharge(curr):
    weakTransformation = 0
    for t in curr: weakTransformation += t[2]
    return weakTransformation

#SU(3)
# loop through colour indices, -1 for lower +1 for higher. Needs to equal 0
def colourCharge(curr):
    strongTransformation = 0
    for t in curr: strongTransformation += t[3]
    return strongTransformation

#Proper handedness
def evenHanded(curr):
    handNum = 0
    for t in curr: handNum += t[6]
    return handNum

def gaugeSymmetry(curr,blocks):
    return hyperCharge(curr,blocks) == 0 and weakCharge(curr) == 0 and colourCharge(curr) == 0

def validTerm(curr,blocks,maxDim):
    return gaugeSymmetry(curr,blocks) and LorentzIndex(curr)==0 and validMass(curr,blocks,maxDim) and evenHanded(curr) == 0
    

In [4]:
validTerm([['h',-1,0,0,0,0,0],['h',1,0,0,0,0,0]],normal,4)

True

In [5]:
# Each term added will be added as a doublet where 
# the second index represents whether (1) or not (0) it has been conjugated - 1/2 if barred
valid = []

# Generate Terms

We will do this a little more carefully. We will perform different operations on the current list based on the field / block that we are adding 

when adding a new field, we will have various indices for different values such as
 - h=1 - {-1,1} - conjugated hypercharge (conjugate + spin 1/2 -> bar)  
 - s=2 - {-1,0,1} - spin index -1 lowered, 1 raised
 - c=3 - {-1,0,1} - colour index 
 - l=4 - {-2,-1,0,1,2} - Lorentz index 
 - a=5 - {0-(len(curr)-1)} - other item being acted on
 
We will be considering the list of terms as an unordered list to prevent duplicates with different orderings. Maybe when we check for duplicates we could also check for h.c to minimize number of terms.

Because of this, we will be raising and lowering the index of the last added term if there is something to contract with. Examples
- if there is an unbarred fermion and we are adding a fermion, then we will bar the one we are adding so that the spin indices contract. 

# Cheat sheet
 Derivative operator - SU(2) 0
 - If there is previously a field to act on, 
 - Fermion - needs slash 
 - Could in principle act on another derivative operator
 - if Lorentz index = 0 then it is slashed and acting on a Fermion. 
 - Could use a way to keep track of what it is acting on. 

 Fermion - spin 1/2
 - check for previous fermion, if found -> bar the previous one and raise/lower spin index
 - 
 
 
# problems to work out. 

- Way to understand what fermion the derivative operator is acting on. Same goes for h.
- Should the covariant derivative operator act on the curr term with the chain rule? Maybe I could treat the derivative operator separate from the fields and then act once per loop using the chain rule
        
        
        


In [25]:
from copy import deepcopy

def addTerm(curr,valid,blocks):
    s = ""
    massDim = 0
    for f in curr:
        if f[0] == 'D' and f[4] == 0:
            s+= f[0] + "_\mu \\gamma^{\mu} " 
        elif f[0] == 'D':
            if f[4] == 1:
                s+= f[0] + "^\mu "
            elif f[4] == -1:
                s+= f[0] + "_\mu "
        else:
            if f[1] == -1: 
                if f[6] != 0:
                    s+= "\\overline{" + f[0] + "} "
                else:
                    s+= "(" + f[0] + ")^{\\dagger} " 
            else: 
                s+= f[0] + " " 
        massDim += blocks[f[0]][0]
    valid.append([str(s) + " + ",massDim])
    return valid

# return list of all terms found by acting covariant derivative operator on 
# current term. First item in list will be unoperated term.
def chainRule(curr,blocks):
    newTerms = []
    newTerms.append(curr)

    for i in range(len(curr)):
        nextTerm = []
        # if term is not derivative operator, act on with chain rule
        if blocks[curr[i][0]] != blocks['D']:
            nextTerm = deepcopy(curr)
            # if operator acting on Fermion
            if blocks[curr[i][0]][4] == 1/2:
                nextTerm.insert(i,['D',0,0,0,0,i+1,-2*curr[i][6]])
            else:
                if LorentzIndex(nextTerm) < 0:
                    nextTerm.insert(i,['D',0,0,0,1,i+1,0])
                else:
                    nextTerm.insert(i,['D',0,0,0,-1,i+1,0])
            
            newTerms.append(nextTerm)
    return newTerms

def termLoop(curr,blocks,maxDim,valid):
    # recursion terminating condition
    if not validMass(curr,blocks,maxDim):
        return 0
    
    # loop through all fields in theory
    for i,key in enumerate(blocks):
        if i > 0: # don't consider covariant derivative in loop - will use chain rule
#           1  h -> hypercharge conjugated
#           2  s -> spinor index
#           3  c -> colour index
#           4  l -> Lorentz index
#           5  a -> acting on (index of object being acted on)
#           6  j -> handedness
#                      h s c l a      j
            add = [key,1,0,0,0,0,blocks[key][5]]

            # get Indices
            
            # Spin index
#             if blocks[key][4] == 1/2: # if Fermion basically 
#                 add[2] = 1
            if blocks[key][2] > 1:
                add[2] = 1
            if blocks[key][3] > 1: # if transforms non trivially under SU(3)
                add[3] = 1
            if blocks[key][4] == 1: # if boson basically - has two lorentz indices
                add[4] = -2

            # contract Indices
            for n in curr:
                #spin indices
                if add[2] == 1 and n[2] == 1:
                    add[2] = -1
                #colour indices
                if add[3] == 1 and n[3] == 1:
                    add[3] = -1
                # lorentz indices
                if add[4] == -2 and n[4] == -2:
                    add[4] = 2
            
            curr.insert(0,add)
            nextList = chainRule(curr,blocks)
            
            for w in nextList:
                if len(w) > 0:
                    if validTerm(w,blocks,maxDim): 
                        valid = addTerm(w,valid,blocks)
                    termLoop(w,blocks,maxDim,valid)
            curr.pop(0)
            
            if blocks[add[0]][1] != 0:
                add[1] *= -1
                curr.insert(0,add)
                nextList = chainRule(curr,blocks)

                for w in nextList:
                    if len(w) > 0:
                        if validTerm(w,blocks,maxDim): 
                            valid = addTerm(w,valid,blocks)
                        termLoop(w,blocks,maxDim,valid)
                curr.pop(0)
    return 0

def generateTerms(blocks):
    cur = []
    valid = []    
    termLoop(cur,blocks,8,valid)
    mini = sorted(valid, key=lambda x: x[1])  
    
    new_k = []
    for elem in mini:
        if elem not in new_k:
            new_k.append(elem)
    
    print(len(new_k))
    
    for t in new_k: 
        print(t[0])
        

In [26]:
valid = []
generateTerms(normal)


65
(h)^{\dagger} h  + 
h (h)^{\dagger}  + 
\overline{u_R^i} q_L^i h  + 
d_R^i \overline{q_L^i} h  + 
q_L^i \overline{u_R^i} h  + 
\overline{q_L^i} d_R^i h  + 
e_R^i \overline{l_L^i} h  + 
\overline{l_L^i} e_R^i h  + 
D^\mu (h)^{\dagger} D_\mu h  + 
(h)^{\dagger} D_\mu D^\mu h  + 
\overline{d_R^i} q_L^i (h)^{\dagger}  + 
u_R^i \overline{q_L^i} (h)^{\dagger}  + 
\overline{q_L^i} u_R^i (h)^{\dagger}  + 
q_L^i \overline{d_R^i} (h)^{\dagger}  + 
\overline{e_R^i} l_L^i (h)^{\dagger}  + 
l_L^i \overline{e_R^i} (h)^{\dagger}  + 
D^\mu h D_\mu (h)^{\dagger}  + 
h D_\mu D^\mu (h)^{\dagger}  + 
\overline{u_R^i} h q_L^i  + 
\overline{d_R^i} (h)^{\dagger} q_L^i  + 
D_\mu \gamma^{\mu} \overline{q_L^i} q_L^i  + 
\overline{q_L^i} D_\mu \gamma^{\mu} q_L^i  + 
h \overline{u_R^i} q_L^i  + 
(h)^{\dagger} \overline{d_R^i} q_L^i  + 
d_R^i h \overline{q_L^i}  + 
u_R^i (h)^{\dagger} \overline{q_L^i}  + 
D_\mu \gamma^{\mu} q_L^i \overline{q_L^i}  + 
q_L^i D_\mu \gamma^{\mu} \overline{q_L^i}  + 
(h)^{\dagger} u

$$(h)^{\dagger} h  + 
h (h)^{\dagger}  + 
\overline{u_R^i} q_L^i h  + 
d_R^i \overline{q_L^i} h  + 
q_L^i \overline{u_R^i} h  + 
\overline{q_L^i} d_R^i h  + 
e_R^i \overline{l_L^i} h  + 
\overline{l_L^i} e_R^i h  + 
D^\mu (h)^{\dagger} D_\mu h  + 
(h)^{\dagger} D_\mu D^\mu h  + 
\overline{d_R^i} q_L^i (h)^{\dagger}  + 
u_R^i \overline{q_L^i} (h)^{\dagger}  + 
\overline{q_L^i} u_R^i (h)^{\dagger}  + 
q_L^i \overline{d_R^i} (h)^{\dagger}  + 
\overline{e_R^i} l_L^i (h)^{\dagger}  + 
l_L^i \overline{e_R^i} (h)^{\dagger}  + 
D^\mu h D_\mu (h)^{\dagger}  + 
h D_\mu D^\mu (h)^{\dagger}  + 
\overline{u_R^i} h q_L^i  + 
\overline{d_R^i} (h)^{\dagger} q_L^i  + 
D_\mu \gamma^{\mu} \overline{q_L^i} q_L^i  + 
\overline{q_L^i} D_\mu \gamma^{\mu} q_L^i  + 
h \overline{u_R^i} q_L^i  + 
(h)^{\dagger} \overline{d_R^i} q_L^i  + 
d_R^i h \overline{q_L^i}  + 
u_R^i (h)^{\dagger} \overline{q_L^i}  + 
D_\mu \gamma^{\mu} q_L^i \overline{q_L^i}  + 
q_L^i D_\mu \gamma^{\mu} \overline{q_L^i}  + 
(h)^{\dagger} u_R^i \overline{q_L^i}  + 
h d_R^i \overline{q_L^i}  + 
\overline{q_L^i} (h)^{\dagger} u_R^i  + 
(h)^{\dagger} \overline{q_L^i} u_R^i  + 
D_\mu \gamma^{\mu} \overline{u_R^i} u_R^i  + 
\overline{u_R^i} D_\mu \gamma^{\mu} u_R^i  + 
q_L^i h \overline{u_R^i}  + 
h q_L^i \overline{u_R^i}  + 
D_\mu \gamma^{\mu} u_R^i \overline{u_R^i}  + 
u_R^i D_\mu \gamma^{\mu} \overline{u_R^i}  + 
\overline{q_L^i} h d_R^i  + 
h \overline{q_L^i} d_R^i  + 
D_\mu \gamma^{\mu} \overline{d_R^i} d_R^i  + 
\overline{d_R^i} D_\mu \gamma^{\mu} d_R^i  + 
q_L^i (h)^{\dagger} \overline{d_R^i}  + 
(h)^{\dagger} q_L^i \overline{d_R^i}  + 
D_\mu \gamma^{\mu} d_R^i \overline{d_R^i}  + 
d_R^i D_\mu \gamma^{\mu} \overline{d_R^i}  + 
\overline{e_R^i} (h)^{\dagger} l_L^i  + 
D_\mu \gamma^{\mu} \overline{l_L^i} l_L^i  + 
\overline{l_L^i} D_\mu \gamma^{\mu} l_L^i  + 
(h)^{\dagger} \overline{e_R^i} l_L^i  + 
e_R^i h \overline{l_L^i}  + 
D_\mu \gamma^{\mu} l_L^i \overline{l_L^i}  + 
l_L^i D_\mu \gamma^{\mu} \overline{l_L^i}  + 
h e_R^i \overline{l_L^i}  + 
\overline{l_L^i} h e_R^i  + 
h \overline{l_L^i} e_R^i  + 
D_\mu \gamma^{\mu} \overline{e_R^i} e_R^i  + 
\overline{e_R^i} D_\mu \gamma^{\mu} e_R^i  + 
l_L^i (h)^{\dagger} \overline{e_R^i}  + 
(h)^{\dagger} l_L^i \overline{e_R^i}  + 
D_\mu \gamma^{\mu} e_R^i \overline{e_R^i}  + 
e_R^i D_\mu \gamma^{\mu} \overline{e_R^i}  + 
G_{\mu \nu} G_{\mu \nu}  + 
W_{\mu \nu} W_{\mu \nu}  + 
B_{\mu \nu} B_{\mu \nu}$$